# 합성곱 신경망(CNN)

CNN(Convolutional Neural Network)은 이미지 인식과 음성 인식 등 다양한 분야에서 사용되는 딥러닝 모델입니다.

 CNN은 이미지의 특징을 추출하기 위해 합성곱층(convolutional layer)과 풀링층(pooling layer)을 사용합니다.

## 7.1. 전체 구조

지금까지의 신경망은 모든 뉴런과 결합되어 있다.

* 이를 **완전연결(fully connected)**이라고 한다.
* 완전 연결된 계층을 **Affine 계층**이라고도 한다.

Affine 계층을 사용하여, 가령 5개의 완전 연결 신경망은 다음과 같은 구조를 갖는다.

```mermaid
graph LR
x[입력]
a1((완전연결계층))
r1((ReLU))
a2((완전연결계층))
r2((ReLU))
a3((완전연결계층))
r3((ReLU))
a4((완전연결계층))
r4((ReLU))
a5((완전연결계층))
s((Softmax))
y((출력))

x --> a1
a1 --> r1
r1 --> a2
a2 --> r2
r2 --> a3
a3 --> r3
r3 --> a4
a4 --> r4
r4 --> a5
a5 --> s
s --> y
```

CNN은 이와 다르게, **합성곱 계층(convolutional layer)**과 **풀링 계층(pooling layer)**을 사용한다.

```mermaid
graph LR
x[입력]
c1((합성곱계층))
r1((ReLU))
p1((풀링계층))
c2((합성곱계층))
r2((ReLU))
p2((풀링계층))
c3((합성곱계층))
r3((ReLU))
a1((완전연결계층))
r4((ReLU))
a2((완전연결계층))
s((Softmax))
y((출력))

x ---> c1
c1 --> r1
r1 --> p1
p1 ---> c2
c2 --> r2
r2 --> p2
p2 ---> c3
c3 --> r3
r3 ---> a1
a1 --> r4
r4 ---> a2
a2 --> s
s --> y
```

CNN에서는 새로운 `합성곱 계층`과 `풀링 계층`이 추가되었다.

CNN계층은 `Conv-ReLU-(Pooling)`으로 연결된다.

## 7.2. 합성곱 계층

CNN에서는 패딩(padding), 스트라이드(stride) 등 CNN 고유의 용어가 등장한다.

또, 각층 사이에는 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.

### 7.2.1. 완전연결 계층의 문제점

완전연결 계층은 데이터의 형상이 무시된다.

입력 데이터가 이미지인 경우, 이미지는 3차원 데이터로 (높이, 너비, 채널)로 구성된다.

그러나 완전연결 계층에 입력할 때는 3차원 데이터를 1차원 데이터로 평탄화(flatten)해야 한다.

이로 인해, 이미지의 공간적인 구조(예를 들어, 가까운 픽셀은 유사하다)가 무시된다.

합성곱 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받아 3차원 데이터로 출력한다.

CNN에서 합성곱 계층의 입출력 데이터를 **특징 맵(feature map)**이라고 한다.

합성곱 계층의 입력 데이터를 **입력 특징 맵**, 출력 데이터를 **출력 특징 맵**이라고 한다.

### 7.2.2. 합성곱 연산

합성곱 연산은 이미지 처리에서 말하는 필터 연산과 같다.

$$
\begin{pmatrix}
1 & 2 & 3 & 0 \\
0 & 1 & 2 & 3 \\
3 & 0 & 1 & 2 \\
2 & 3 & 0 & 1 \\
\end{pmatrix}

*
\begin{pmatrix}
2 & 0 & 1 \\
0 & 1 & 2 \\
1 & 0 & 2 \\
\end{pmatrix}

= 
\begin{pmatrix}
15 & 16 \\
9 & 15 \\
\end{pmatrix}
$$

합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용한다.

만약 (4, 4) 크기의 입력 데이터에 (3, 3) 크기의 필터를 적용하여 (2,2) 크기의 출력데이터를 얻을 수 있다.

편향을 더해주면, 다음과 같은 식으로 나타낼 수 있다.

$$
a_{x,y}^{(l)} = \sum_{i=0}^{f-1} \sum_{j=0}^{f-1} \sum_{k=0}^{c-1} w_{i,j,k}^{(l)} x_{x+i, y+j, k} + b^{(l)}
$$

여기서, $a_{x,y}^{(l)}$은 출력 데이터, $w_{i,j,k}^{(l)}$은 필터의 가중치, $x_{x+i, y+j, k}$는 입력 데이터, $b^{(l)}$은 편향이다.



### 7.2.3 패딩

합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(0)으로 채우는 것을 **패딩**이라고 한다.

패딩은 주로 출력 데이터의 크기를 조정할 목적으로 사용된다.

예를 들어 (4, 4) 크기의 입력 데이터에 (3, 3) 크기의 필터를 적용하면 (2, 2) 크기의 출력 데이터를 얻는다.

하지만, 합성곱 연산을 여러번 수행하면, 출력 데이터의 크기가 작아진다. 1이 되면 더 이상 합성곱 연산을 수행할 수 없다.

이를 방지하기 위해 패딩을 사용한다.

### 7.2.4. 스트라이드

필터를 적용하는 위치의 간격을 **스트라이드**라고 한다. 지금까지 본 예는 스트라이드가 1이었다.

스트라이드를 2로 하면 필터가 이동하는 간격이 2가 된다.